In [4]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("../data_given/winequality.csv")

In [3]:
overview = df.describe()

In [7]:
overview.loc[["min","max"]].to_json("schema_in.json")

In [9]:
class NotInRange(Exception):
    def __init__(self, message="value not in range"):
#         self.input_ = input_
        self.message = message
        super().__init__(self.message)

In [10]:
raise NotInRange

NotInRange: value not in range

In [5]:
pwd

'/home/bigpenguin/projects/mlops/notebooks'

In [6]:
os.chdir('/home/bigpenguin/projects/mlops')

In [8]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]

    model_name = mlflow_config["registered_model_name"]

    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_registry_uri(remote_server_uri)

    runs = mlflow.search_runs(experiment_ids='1')
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mc in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mc)
        pprint(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv,indent=4)

            client.transition_model_version_stage(
                name = model_name,
                version = current_version,
                stage = "Production"
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name = model_name,
                version = current_version,
                stage = "Staging"
            )
    
    print(logged_model)


log_production_model("params.yaml")

# if __name__=='__main__':
#     args = argparse.ArgumentParser()
#     args.add_argument("--config",default="params.yaml")
#     parsed_args = args.parse_args()
#     data = log_production_model(config_path=parsed_args.config)

{'creation_timestamp': 1676222367542,
 'current_stage': 'Staging',
 'description': '',
 'last_updated_timestamp': 1676283900976,
 'name': 'ElasticNetWineModel',
 'run_id': '07f5d73b80804f4692ae3ada155bd307',
 'run_link': '',
 'source': './artifacts/1/07f5d73b80804f4692ae3ada155bd307/artifacts/model',
 'status': 'READY',
 'status_message': '',
 'tags': {},
 'user_id': '',
 'version': '2'}
{'creation_timestamp': 1676217174663,
 'current_stage': 'Staging',
 'description': '',
 'last_updated_timestamp': 1676283900954,
 'name': 'ElasticNetWineModel',
 'run_id': '1cff9eaa607e4e31889ef1e26d1c195a',
 'run_link': '',
 'source': './artifacts/1/1cff9eaa607e4e31889ef1e26d1c195a/artifacts/model',
 'status': 'READY',
 'status_message': '',
 'tags': {},
 'user_id': '',
 'version': '1'}


UnboundLocalError: local variable 'logged_model' referenced before assignment